In [20]:
from langchain_ollama import ChatOllama

mistral = ChatOllama(
    model="mistral",
    temperature=0,
)

In [21]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    model="mistral",
)

In [22]:
import getpass
import os
import time

from pinecone import Pinecone, ServerlessSpec

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [23]:
import time

index_name = "personal" 

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=4096,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [24]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [25]:
mistral_retriever = vector_store.as_retriever(search_kwargs={'k':2})

In [26]:
from langchain_core.tools import tool

@tool
def marks_reader(self, file_path: str="nirajan_transcript.txt") -> str:
    """Read content from a text file."""
    try:
        if not os.path.exists(file_path):
            return f"Error: File not found at path {file_path}"
        
        with open(file_path, 'r', encoding='utf-8') as file:
            content = file.read()
        
        return content
    except Exception as e:
        return f"Error reading file: {str(e)}"
    
tools = [marks_reader]
mistral_with_tools = mistral.bind_tools(tools)

In [27]:
from langchain.prompts import PromptTemplate

template = """
    You are a helpful personal assistant of this user that can provide information based on both vector search from a database and other tools.

    ### Task:
    You will be provided with a user query. Your goal is to respond to the query by doing the following:
    1. Retrieve relevant information from the vector database (Pinecone) that best matches the query.
    2. If additional context or information is required that can be retrieved from tools, use the appropriate tool.
    3. Combine the information from the database and tools to provide a well-structured and complete response.

    ### User Query:
    {user_query}

    ### Relevant Documents (from Pinecone vector database):
    {retrieved_documents}

    ### Tool Usage:
    If the relevant documents or context are not sufficient to answer the query, use the tools you have. For example, if the user is asking for marks of certain subjects, use the marks_reader tool to extract information from the document.
    If you used any tools, describe how the tool was used in your response.

    ### Answer:
    Your answer should be concise, clear, and comprehensive, using the retrieved documents and any tool-assisted information.
 """

prompt = PromptTemplate(
    input_variables=["user_query", "retrieved_documents"],
    template=template,
)

In [ ]:
chain = prompt | mistral_with_tools

In [ ]:
# user_query = input("Your query :: \n")
# retrieved_documents = mistral_retriever.invoke(user_query)

# chain = prompt | mistral_with_tools

# response = chain.invoke({"user_query": user_query, "retrieved_documents": retrieved_documents})
# print(response.content)

In [ ]:
# print(response.content)

 The name of the candidate mentioned in the provided document is Nabaraj Subedi.


### Let's add memory to the chatbot using **langgraph**

In [54]:
from typing import TypedDict, Annotated, Sequence
from langgraph.graph import START, END, StateGraph
import operator

class GraphState(TypedDict):
    user_query : str
    retrieved_documents : Sequence[str]
    response : str

def input_node(state: GraphState) -> GraphState:
    return state

def retrieval_node(state: GraphState)->GraphState:
    retrieved_documents = mistral_retriever.invoke(state['user_query'])
    state['retrieved_documents'] = retrieved_documents
    return state

def processing_node(state: GraphState)->GraphState:
    chain = prompt | mistral_with_tools
    response = chain.invoke(
        {
            "user_query": state['user_query'],
            "retrieved_documents": state['retrieved_documents']
        }
    )
    state['response'] = response.content
    return state

workflow = StateGraph(GraphState)

workflow.add_node("input", input_node)
workflow.add_node("retrieval", retrieval_node)
workflow.add_node("processing", processing_node)

workflow.add_edge(START, "input")
workflow.add_edge("input","retrieval")
workflow.add_edge("retrieval","processing")
workflow.add_edge("processing",END)

from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

app = workflow.compile(checkpointer=memory)

In [55]:
config = {"configurable": {"thread_id": "1"}}

In [58]:
user_query = input("YOU:: \n")
initial_state = {"user_query": user_query}
result = app.invoke(initial_state,config=config)
print(result['response'])

 Based on the documents I have retrieved, it appears that there is no direct question or answer related to your previous question in the provided text files. However, if you are referring to a specific topic or subject discussed in these documents, I can provide more information about them.

   For instance, one of the documents discusses a paper titled "Bornon: Bengali image captioning with transformer-based deep learning approach" by Muhammad Shah et al., published in SN Computer Science 3 (2022). If you are referring to this paper or its topic, I can provide more details about it.

   To get more specific information, I could use the marks_reader tool to search for any personal documents that might contain information related to your previous question. However, since no such documents were provided in the context of this query, I will not use the marks_reader tool at this time. If you have a specific document or topic in mind, please let me know so I can help you find more informati